In [598]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")



In [599]:
df = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [600]:
df.columns

Index(['id', 'scrape_id', 'last_scraped', 'name', 'description', 'picture_url',
       'host_id', 'host_name', 'host_since', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listi

In [601]:
import collections


In [602]:
df['ratings'] = df['name'].apply(lambda x: find_keyword(x, '\u2605', 1, 4))
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
for value in df['ratings']:
    print(value)

nan
4.74
5.0
nan
nan
4.94
3.0
nan
4.83
nan
4.0
4.67
nan
nan
nan
5.0
nan
nan
nan
nan
nan
4.97
5.0
nan
nan
4.88
nan
4.7
3.5
4.53
nan
nan
nan
nan
4.59
nan
nan
4.92
4.77
4.74
nan
nan
nan
nan
4.74
5.0
nan
nan
4.46
nan
4.6
nan
5.0
nan
5.0
4.77
nan
5.0
4.95
5.0
4.94
4.83
nan
4.98
4.9
nan
nan
nan
5.0
nan
4.33
5.0
4.96
5.0
nan
nan
4.95
nan
4.94
4.89
4.96
nan
nan
nan
4.96
4.96
nan
nan
4.73
nan
nan
nan
4.86
4.86
nan
4.87
4.6
5.0
4.97
nan
nan
4.83
4.5
nan
4.78
nan
nan
5.0
nan
nan
nan
nan
4.97
4.43
4.93
4.67
4.82
nan
4.67
nan
nan
4.86
nan
4.29
nan
4.96
5.0
nan
4.96
4.92
4.8
4.5
4.88
4.6
4.51
4.67
nan
5.0
nan
5.0
4.86
3.67
nan
4.81
5.0
nan
nan
nan
5.0
4.82
5.0
nan
nan
nan
nan
4.2
nan
4.85
2.33
4.81
nan
5.0
nan
4.5
nan
nan
nan
4.8
4.85
nan
5.0
4.33
nan
5.0
4.87
nan
nan
nan
nan
4.86
4.9
4.95
4.2
4.71
4.58
4.98
4.43
4.6
nan
4.94
5.0
nan
4.76
4.81
4.5
5.0
4.9
nan
5.0
nan
nan
4.86
4.9
nan
4.64
3.67
3.5
nan
5.0
5.0
nan
nan
nan
4.9
4.6
4.5
4.63
4.8
4.97
4.9
nan
4.89
nan
nan
4.81
nan
4.87
nan
nan
4.7
nan
4.

In [603]:
# Convert continuous ratings to numeric, handling errors by setting them to NaN
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')

# Create quintiles for continuous ratings and one-hot encode them
# Separate NaN and non-NaN values
non_nan_values = df['ratings'].dropna()

# Create quintiles for non-NaN values
df.loc[~df['ratings'].isna(), 'quintile'] = pd.qcut(non_nan_values, q=4, labels=False, precision=1)

# Handle NaN values (assign NaN to quintile column)
df.loc[df['ratings'].isna(), 'quintile'] = None

# Generate dummy variables
df = pd.get_dummies(df, columns=['quintile'], prefix='rating', dummy_na=True)

In [604]:
def get_unique_words(df, column_name):
    x = collections.Counter([y for x in df.loc[:,column_name].astype(str).values.flatten() for y in x.lower().replace('[', '').replace(']', '').replace(' ',"").replace('\'',"").replace('"',"").split(",")])
    return x

In [605]:
unique_amenities = get_unique_words(df, 'amenities')

In [606]:
# unique_amenities.most_common(100)

In [607]:
def find_keyword(text, keyword):
    index = text.find(keyword)
    if index != -1 and index >= 2:
        return text[index - 2]
    else:
        return None


In [608]:
def find_keyword(text, keyword, location=-2, length=1):
    index = text.find(keyword)
    if index == -1:
        return None
    try:
        return text[index + location:index + location + length]
    except:
        return None


In [609]:
df['name'].apply(lambda x: find_keyword(x, 'bedroom'))

0        1
1        2
2        3
3        4
4        1
        ..
15436    2
15437    4
15438    1
15439    1
15440    2
Name: name, Length: 15441, dtype: object

In [610]:
latest = pd.to_datetime(df['host_since']).sort_values().iloc[-1]

In [611]:
(latest - pd.to_datetime(df['host_since'])).dt.days

0        1861
1        3695
2        1529
3         668
4         192
         ... 
15436    1085
15437    1881
15438    2883
15439    1611
15440    2914
Name: host_since, Length: 15441, dtype: int64

In [612]:
import re

In [613]:
df['bathrooms_text'].astype(str).str.extract(r'(\d+(\.\d+)?)')[0]

0          1
1          2
2          2
3        2.5
4          1
        ... 
15436      2
15437    3.5
15438      1
15439    NaN
15440      2
Name: 0, Length: 15441, dtype: object

In [614]:
def cleanData(df, test):

    ## Find Stars for Ratings
    df['ratings'] = df['name'].apply(lambda x: find_keyword(x, '\u2605', 1, 4))
    test['ratings'] = test['name'].apply(lambda x: find_keyword(x, '\u2605', 1, 4))

    # Create dummy columns for 'new' and 'none'
    df['new_rating'] = df['ratings'].apply(lambda x: 1 if x == 'New ' else 0)
    test['new_rating'] = test['ratings'].apply(lambda x: 1 if x == 'New ' else 0)

    # Convert continuous ratings to numeric, handling errors by setting them to NaN
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
    test['ratings'] = pd.to_numeric(test['ratings'], errors='coerce')

    # Create quintiles for continuous ratings and one-hot encode them
    # Separate NaN and non-NaN values
    non_nan_values = df['ratings'].dropna()

    # Create quintiles for non-NaN values
    quintiles, cutoffs = pd.qcut(non_nan_values, q=3, labels=False, retbins=True, precision=1)
    df.loc[~df['ratings'].isna(), 'quintile'] = pd.cut(non_nan_values, bins=cutoffs, labels=False)
    test.loc[~test['ratings'].isna(), 'quintile'] = pd.cut(non_nan_values, bins=cutoffs, labels=False)

    # Handle NaN values (assign NaN to quintile column)
    df.loc[df['ratings'].isna(), 'quintile'] = None
    test.loc[test['ratings'].isna(), 'quintile'] = None

    # Convert quintile column to integer type
    df['quintile'] = df['quintile'].astype(pd.Int64Dtype())
    test['quintile'] = test['quintile'].astype(pd.Int64Dtype())

    # Generate dummy variables
    df = pd.get_dummies(df, columns=['quintile'], prefix='rating', dummy_na=True)
    df = df.drop("ratings", axis=1)
    test = pd.get_dummies(test, columns=['quintile'], prefix='rating', dummy_na=True)
    test = test.drop("ratings", axis=1)

    ## Change Date for Host Since
    latest = pd.to_datetime(df['host_since']).sort_values().iloc[-1]
    df["host_days"] = (latest - pd.to_datetime(df['host_since'])).dt.days
    df = df.drop("host_since", axis=1)
    latest = pd.to_datetime(test['host_since']).sort_values().iloc[-1]
    test["host_days"] = (latest - pd.to_datetime(test['host_since'])).dt.days
    test = test.drop("host_since", axis=1)

    ## Find Bathrooms
    df["num_bathrooms"] = df['bathrooms_text'].astype(str).str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
    df = df.drop("bathrooms_text", axis=1)
    test["num_bathrooms"] = test['bathrooms_text'].astype(str).str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
    test = test.drop("bathrooms_text", axis=1)

    ## Find Bedrooms
    df["num_bedrooms"] = df['name'].apply(lambda x: find_keyword(x, 'bedroom')).astype(float)
    df = df.drop("name", axis=1)
    test["num_bedrooms"] = test['name'].apply(lambda x: find_keyword(x, 'bedroom')).astype(float)
    test = test.drop("name", axis=1)

    ## Categorise Amenities
    amenities_list = get_unique_words(df, 'amenities').most_common(100)
    df['amenities'] = df['amenities'].astype('string')
    test['amenities'] = test['amenities'].astype('string')

    df['amenities'] = df['amenities'].apply(lambda x: x.lower().replace('[', '').replace(']', '').replace(' ', '').replace('"', ''))
    test['amenities'] = test['amenities'].apply(lambda x: x.lower().replace('[', '').replace(']', '').replace(' ', '').replace('"', ''))

    string_amenities = [amenity[0] for amenity in amenities_list if isinstance(amenity[0], str)]
    for amenity in string_amenities:
        df[amenity] = df['amenities'].apply(lambda x: 1 if amenity.lower() in x.lower() else 0)
        test[amenity] = test['amenities'].apply(lambda x: 1 if amenity.lower() in x.lower() else 0)
    df = df.drop("amenities", axis=1)
    test = test.drop("amenities", axis=1)

    # Categorise host verifications
    verif_list = ['email', 'phone', 'work_email']
    for verif in verif_list:
        df[verif] = df['host_verifications'].apply(lambda x: 1 if verif in x else 0)
        test[verif] = test['host_verifications'].apply(lambda x: 1 if verif in x else 0)
    df = df.drop("host_verifications", axis=1)
    test = test.drop("host_verifications", axis=1)

    # converting false and true to binary
    df.replace({'f': 0, 't': 1}, inplace=True)
    test.replace({'f': 0, 't': 1}, inplace=True)

    # dropping all useless variables
    to_drop = ["host_name", "host_id", 'scrape_id', 'last_scraped', 'description', 'picture_url','latitude',
       'longitude','calendar_last_scraped']
    df = df.drop(to_drop, axis=1)
    test = test.drop(to_drop, axis=1)

    ## dummify all categorical variables
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns, prefix=categorical_columns, dummy_na=False)
    test = pd.get_dummies(test, columns=categorical_columns, prefix=categorical_columns, dummy_na=False)

    df.rename(columns={'rating_<NA>': 'none_rating'}, inplace=True)
    test.rename(columns={'rating_<NA>': 'none_rating'}, inplace=True)


    df, test = df.align(test, axis=1, fill_value=0)
    test = test.drop("price", axis=1)
    
    return df, test

In [615]:
cleaned_train, cleaned_test = cleanData(train, test)

In [616]:
cleaned_train.to_csv("cleaned_train.csv", index=False)
cleaned_test.to_csv("cleaned_test.csv", index=False)

In [617]:
cleaned_train['id']

0            0
1            1
2            2
3            3
4            4
         ...  
15434    15434
15435    15435
15436    15436
15438    15438
15440    15440
Name: id, Length: 14029, dtype: int64

In [618]:
for col in cleaned_train.columns:
    print(col)
print(len(cleaned_train.columns))
print(len(cleaned_train.columns))

accommodates
airconditioning
availability_30
availability_365
availability_60
availability_90
backyard
bakingsheet
barbecueutensils
bathtub
bbqgrill
beachessentials
bedlinens
beds
blender
boardgames
bodysoap
booksandreadingmaterial
calculated_host_listings_count
calculated_host_listings_count_entire_homes
calculated_host_listings_count_private_rooms
calculated_host_listings_count_shared_rooms
carbonmonoxidealarm
ceilingfan
centralairconditioning
centralheating
children\u2019sdinnerware
cityskylineview
cleaningavailableduringstay
cleaningproducts
clothingstorage
clothingstorage:closet
coffee
coffeemaker
coffeemaker:dripcoffeemaker
coffeemaker:keurigcoffeemachine
conditioner
cookingbasics
crib
dedicatedworkspace
diningtable
dishesandsilverware
dishwasher
dryer
dryingrackforclothing
elevator
email
essentials
ethernetconnection
exerciseequipment
extrapillowsandblankets
fireextinguisher
firepit
firstaidkit
freedryer\u2013inunit
freeparkingonpremises
freestreetparking
freewasher\u2013inunit
